In [3]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [4]:
pickle_file= 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save=pickle.load(f)
    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset']
    test_labels=save['test_labels']
    del save #hint to help gc to free up memory 
    print('training set', train_dataset.shape,train_labels.shape)
    print('validation set', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape,test_labels.shape)
    

training set (200000, 28, 28) (200000,)
validation set (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [5]:
image_size=28
num_labels=10

def reformat(dataset, labels):
    dataset=dataset.reshape((-1,image_size*image_size)).astype(np.float32)
    # Map 1 to [0.0,1.0,0.0....], 2 to [0.0,0.0,1.0.....]
    labels=(np.arange(num_labels) ==labels[:,None]).astype(np.float32)
    return dataset,labels
train_dataset, train_labels= reformat(train_dataset, train_labels)
valid_dataset, valid_labels=reformat(valid_dataset, valid_labels)
test_dataset, test_labels =reformat(test_dataset, test_labels)
print( 'training set', train_dataset.shape,train_labels.shape)
print('validation set', valid_dataset.shape,valid_labels.shape)
print('test set', test_dataset.shape,test_labels.shape)

training set (200000, 784) (200000, 10)
validation set (10000, 784) (10000, 10)
test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
    return(100.0*np.sum(np.argmax(predictions, 1)==np.argmax(labels,1))/ predictions.shape[0])

In [7]:
log_dir='Hidden_2layer_dropout_regularize__decay_log'

def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
    tf.gfile.MakeDirs(log_dir)

In [2]:
batch_size=128 #sgd minibatch size
num_nodes=1024 
hidden_nodes_2=int(num_nodes*0.5)
SGDSTEP=0.5
beta=0.01 #regularizer constant/learning rate
#NumberofLayers=2 #(front layer+1 hidden layer)

sess = tf.InteractiveSession()    
graph=tf.Graph()
with graph.as_default():
    # Input data.For the training data,we use a placeholder to fed with a training mini batch
    #at runtime with a training min i batch
    tf_train_dataset= tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #layer 1 weight and bias
    with tf.name_scope('weights1'):
        weights_1=tf.Variable(tf.truncated_normal([image_size*image_size,num_nodes]))
        ##variable_summaries(weights_1)
    with tf.name_scope('biases1'):
        biases_1=tf.Variable(tf.zeros([num_nodes]))
        ##variable_summaries(biases_1)
        
    #layer 2 weight and bias
    with tf.name_scope('weights2'):
        weights_2=tf.Variable(tf.truncated_normal([num_nodes, hidden_nodes_2]))
        ##variable_summaries(weights_2)
    with tf.name_scope('biases2'):
        biases_2=tf.Variable(tf.zeros(hidden_nodes_2))
        ##variable_summaries(biases_2)
    
    #output layer weight and bias
    with tf.name_scope('weights3'):
        weights_3=tf.Variable(tf.truncated_normal([hidden_nodes_2,num_labels]))
        ##variable_summaries(weights_3)
    with tf.name_scope('biases3'):
        biases_3=tf.Variable(tf.zeros(num_labels))
        ##variable_summaries(biases_3)
    
    #Logits1 with relu1
    with tf.name_scope('W1x_plus_b1'):
        preactivate = tf.matmul(tf_train_dataset,weights_1)+ biases_1
        ##tf.summary.histogram('pre_activations1', preactivate)
    relu_layer_1=tf.nn.relu(preactivate)
    
    #dropuout1
    drop_prob= tf.placeholder("float")
    relu_layer_1_dropout=tf.nn.dropout(relu_layer_1, drop_prob)
    ##tf.summary.histogram('activations', relu_layer_1_dropout)
    
    #Logits2 with relu2
    preactivate_2 = tf.matmul(relu_layer_1_dropout,weights_2)+ biases_2
    ##tf.summary.histogram('pre_activations1', preactivate_2)
    relu_layer_2=tf.nn.relu(preactivate_2)
    #dropuout2
    
    relu_layer_2_dropout=tf.nn.dropout(relu_layer_2, drop_prob)
    ##tf.summary.histogram('activations', relu_layer_2_dropout)
    
    #Output Logits
    
    logits=tf.matmul(relu_layer_2_dropout,weights_3)+biases_3
    #loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    
    with tf.name_scope('cross_entropy'):
        # The raw formulation of cross-entropy,
        #
        # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
        #                               reduction_indices=[1]))
        #
        # can be numerically unstable.
        #
        # So here we use tf.nn.softmax_cross_entropy_with_logits on the
        # raw outputs of the nn_layer above, and then average across
        # the batch.
        with tf.name_scope('total'):
            #normal loss function
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
            #loss function with l2 regularaization with beta 0.01
            regularizers=tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)+tf.nn.l2_loss(weights_3)
            loss=tf.reduce_mean(loss+beta*regularizers)
    tf.summary.scalar('cross_entropy', loss)

    
    #optimizer
    with tf.name_scope('train'):
        global_step=tf.Variable(0) #count the number of steps taken
        start_learning_rate=0.5
        ##learning_rate=tf.train.exponential_decay(start_learning_rate,global_step,100000,0.96,staircase=True)
        optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
    #predictions for training
    train_prediction=tf.nn.softmax(logits)
    
    #Validation layer1
    valid_logit1=tf.matmul(tf_valid_dataset, weights_1)+biases_1
    valid_relu1=tf.nn.relu(valid_logit1)
    
    #Validation layer2
    valid_logit2=tf.matmul(valid_relu1, weights_2)+biases_2
    valid_relu2=tf.nn.relu(valid_logit2)
    
    #Validation layer3
    valid_logit3=tf.matmul(valid_relu2, weights_3)+biases_3
    
    valid_prediction=tf.nn.softmax(valid_logit3)
    
    
    #Test layer1
    test_logit1=tf.matmul(tf_test_dataset, weights_1)+biases_1
    test_relu1=tf.nn.relu(test_logit1)
    
    #Test layer2
    test_logit2=tf.matmul(test_relu1, weights_2)+biases_2
    test_relu2=tf.nn.relu(test_logit2)
    
    #Test layer2
    test_logit3=tf.matmul(test_relu2, weights_3)+biases_3
    
    test_prediction=tf.nn.softmax(test_logit3)
    
    
   
    
   
    print(graph)
    print(sess)

NameError: name 'tf' is not defined

In [1]:
num_steps=30001

with tf.Session(graph=graph) as sess:
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(log_dir + '/train',sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    tf.global_variables_initializer().run()
    print("Initialized")
    c=0
    
    for step in range(num_steps):
        #Pick an offset within taining data. which has been randomized.
        # Note: we could use better randomization across epochs
        offset= (step*batch_size) % (train_labels.shape[0] - batch_size) # we have used train_labels.shape[0] it is equal to the toatal no of image in the datasets
        
        #generate a minibatch , as SGD requires them
        batch_data= train_dataset[offset:(offset+batch_size),:]
        batch_labels= train_labels[offset:(offset+batch_size),:]
        
        #Prepare a dictionary telling the session where to feed the minibatch
        # The Key of the dictinary is the placeholder node of the graph to be fed,
        #and the value is the numpy array to feed to it.
        feed_dict={tf_train_dataset: batch_data,tf_train_labels: batch_labels,drop_prob:0.5}
        _,summary, l,predictions = sess.run( [optimizer,merged,loss, train_prediction], feed_dict=feed_dict)
        test_writer.add_summary(summary, step)
        
        if (step % 500==0):
            miniAccuracy=accuracy( valid_prediction.eval(), valid_labels)
            print("Minibatch loss at step %d: %f" % (step,l))
            print("Minibatch accurracy: %.1f%%" % miniAccuracy)
            ##print (c)
            ##step_array.append(step)
            ##accuracy_array.append(miniAccuracy)
            ##c=c+1
            ##plt.plot(step,miniAccuracy)
        print("Test Accuracy: %.1f%%" % accuracy(test_prediction.eval(),test_labels))
        
        ##print(accuracy(test_predicton.eval(),test_labels))
        
sess.close()

NameError: name 'tf' is not defined